In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# 示例DataFrame
data = {
    'OD': ['A-B', 'A-C', 'B-C', 'B-D', 'C-D'],
    'Distance': [10, 20, 15, 25, 30]
}
df = pd.DataFrame(data)

def polynomial_probability(d, coeffs):
    a, b, c = coeffs
    p = a * d**2 + b * d + c
    P = np.clip(p, 0, 1)
    return P

def run_abm(probabilities):
    """
    示例ABM模型：得分为概率的加权和。
    请替换为实际的ABM模型实现。
    """
    # 示例权重，可以根据实际情况调整
    weights = np.ones_like(probabilities)
    score = np.dot(probabilities, weights)
    return score

def objective_with_penalty(coeffs, distances, target_score=85, penalty_weight=1000):
    probabilities = polynomial_probability(distances, coeffs)
    score = run_abm(probabilities)
    penalty = np.sum(np.maximum(probabilities - 1.0, 0))
    return (target_score - score)**2 + penalty_weight * penalty

# 提取距离数组
distances = df['Distance'].values

# 初始多项式系数，[a, b, c]
initial_coeffs = [0.0, 0.0, 0.1]

# 定义边界
bounds = [(-1e-4, 1e-4), (-1e-2, 1e-2), (0, 1)]

# 运行优化
result = minimize(
    objective_with_penalty,
    initial_coeffs,
    args=(distances,),
    method='L-BFGS-B',
    bounds=bounds,
    options={'disp': True, 'maxiter': 1000}
)

# 检查优化结果
if result.success:
    optimized_coeffs = result.x
    print(f"优化成功！最优系数: {optimized_coeffs}")
    final_probabilities = polynomial_probability(distances, optimized_coeffs)
    final_score = run_abm(final_probabilities)
    print(f"最终得分: {final_score}")
    print(f"最终概率值: {final_probabilities}")
else:
    print("优化失败。")
    print(result.message)
